In [ ]:
import os
os.environ["KERAS_BACKEND"] = "tensorflow"

In [ ]:
import keras
print(keras.__version__)

3.8.0


In [ ]:
import numpy as np
import pandas as pd

In [ ]:
# downloading the dataset from Kaggle
!kaggle datasets download -d ikarus777/best-artworks-of-all-time -p ../Data

Dataset URL: https://www.kaggle.com/datasets/ikarus777/best-artworks-of-all-time
License(s): CC-BY-NC-SA-4.0
100% 2.29G/2.29G [00:28<00:00, 73.6MB/s]
100% 2.29G/2.29G [00:28<00:00, 86.6MB/s]


In [ ]:
# unzipping the dataset
!unzip ../Data/best-artworks-of-all-time.zip -d ../Data

In [ ]:
pip install split-folders

In [ ]:
# splitting the dataset into training and test sets using the splitfolders module
import splitfolders

input_folder = "/Data/images/images"
splitfolders.ratio(input_folder, output="output_folder", seed=42, ratio=(0.8, 0.2))

Copying files: 8774 files [00:20, 423.07 files/s]


In [ ]:
# reading image paths and labels from class directories and creating a dataframeگ
import os
import pandas as pd

data_path = "/Data/images/images/"

image_paths = []
labels = []

for class_name in os.listdir(data_path):
    class_path = os.path.join(data_path, class_name)

    if os.path.isdir(class_path):
        for img_name in os.listdir(class_path):
            if img_name.endswith(('.jpg', '.png', '.jpeg')):
                image_paths.append(os.path.join(class_path, img_name))
                labels.append(class_name)

df_train = pd.DataFrame({'full_path': image_paths, 'painter': labels})

df_train

,full_path,painter
0,/Data/images/images/Vasiliy_Kandinskiy/Vasiliy...,Vasiliy_Kandinskiy
1,/Data/images/images/Vasiliy_Kandinskiy/Vasiliy...,Vasiliy_Kandinskiy
2,/Data/images/images/Vasiliy_Kandinskiy/Vasiliy...,Vasiliy_Kandinskiy
3,/Data/images/images/Vasiliy_Kandinskiy/Vasiliy...,Vasiliy_Kandinskiy
4,/Data/images/images/Vasiliy_Kandinskiy/Vasiliy...,Vasiliy_Kandinskiy
...,...,...
8769,/Data/images/images/Jan_van_Eyck/Jan_van_Eyck_...,Jan_van_Eyck
8770,/Data/images/images/Jan_van_Eyck/Jan_van_Eyck_...,Jan_van_Eyck
8771,/Data/images/images/Jan_van_Eyck/Jan_van_Eyck_...,Jan_van_Eyck
8772,/Data/images/images/Jan_van_Eyck/Jan_van_Eyck_...,Jan_van_Eyck


In [ ]:
class_names = df_train.painter.unique()
print(class_names)

['Vasiliy_Kandinskiy' 'Hieronymus_Bosch' 'Henri_de_Toulouse-Lautrec'
 'Albrecht_Du╠Иrer' 'Peter_Paul_Rubens' 'Michelangelo' 'Marc_Chagall'
 'Rembrandt' 'Kazimir_Malevich' 'Diego_Rivera' 'Raphael' 'Frida_Kahlo'
 'Albrecht_DuтХа├кrer' 'Piet_Mondrian' 'Claude_Monet' 'Amedeo_Modigliani'
 'Sandro_Botticelli' 'Mikhail_Vrubel' 'Gustave_Courbet' 'El_Greco'
 'Andy_Warhol' 'Pierre-Auguste_Renoir' 'Leonardo_da_Vinci' 'Alfred_Sisley'
 'Henri_Matisse' 'Joan_Miro' 'Paul_Cezanne' 'Paul_Klee' 'Henri_Rousseau'
 'Edgar_Degas' 'Caravaggio' 'Gustav_Klimt' 'Rene_Magritte'
 'Pieter_Bruegel' 'Georges_Seurat' 'Francisco_Goya' 'Andrei_Rublev'
 'Diego_Velazquez' 'Titian' 'Paul_Gauguin' 'Edouard_Manet' 'Salvador_Dali'
 'Pablo_Picasso' 'Vincent_van_Gogh' 'Giotto_di_Bondone' 'Eugene_Delacroix'
 'William_Turner' 'Camille_Pissarro' 'Edvard_Munch' 'Jackson_Pollock'
 'Jan_van_Eyck']


In [ ]:
# displaying random images from the dataset with their corresponding labels
import matplotlib.pyplot as plt

fig, axes = plt.subplots(3, 4, figsize=(15, 15))
random_indices = np.random.choice(df_train.index, size=12, replace=False)
for i, ax in zip(random_indices, axes.flatten()):
    img = keras.preprocessing.image.load_img(df_train.full_path[i], target_size=(224, 224))
    ax.imshow(img)
    ax.set_title(df_train.painter[i])
    ax.axis('off')
plt.tight_layout()
plt.show()

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
# encoding categorical labels into numeric values using LabelEncoder
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
df_train['label'] = label_encoder.fit_transform(df_train.painter)
df_train.head()

,full_path,painter,label
0,/Data/images/images/Vasiliy_Kandinskiy/Vasiliy...,Vasiliy_Kandinskiy,48
1,/Data/images/images/Vasiliy_Kandinskiy/Vasiliy...,Vasiliy_Kandinskiy,48
2,/Data/images/images/Vasiliy_Kandinskiy/Vasiliy...,Vasiliy_Kandinskiy,48
3,/Data/images/images/Vasiliy_Kandinskiy/Vasiliy...,Vasiliy_Kandinskiy,48
4,/Data/images/images/Vasiliy_Kandinskiy/Vasiliy...,Vasiliy_Kandinskiy,48


In [ ]:
print(df_train['label'].unique().min())
print(df_train['label'].unique().max())

0
50


In [ ]:
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 64
NUM_CLASSES = len(class_names)

In [ ]:
# loading and preparing training and validation datasets from a directory using image_dataset_from_directory
from keras.utils import image_dataset_from_directory

train_dataset = image_dataset_from_directory(
    directory='/Data/images/images/',
    labels='inferred',
    label_mode='categorical',
    color_mode='rgb',
    batch_size=BATCH_SIZE,
    image_size=IMAGE_SIZE,
    shuffle=True,
    seed=42,
    validation_split=0.2,
    subset='training'
)

# finding the class names so in prediction time we can map the predictions to the painters properly
class_names = train_dataset.class_names
print('Class names:', class_names)

val_dataset = image_dataset_from_directory(
    directory='/Data/images/images/',
    labels='inferred',
    label_mode='categorical',
    color_mode='rgb',
    batch_size=BATCH_SIZE,
    image_size=IMAGE_SIZE,
    shuffle=False,
    seed=42,
    validation_split=0.2,
    subset='validation'
)

Found 8774 files belonging to 51 classes.
Using 7020 files for training.
Class names: ['Albrecht_DuтХа├кrer', 'Albrecht_Du╠Иrer', 'Alfred_Sisley', 'Amedeo_Modigliani', 'Andrei_Rublev', 'Andy_Warhol', 'Camille_Pissarro', 'Caravaggio', 'Claude_Monet', 'Diego_Rivera', 'Diego_Velazquez', 'Edgar_Degas', 'Edouard_Manet', 'Edvard_Munch', 'El_Greco', 'Eugene_Delacroix', 'Francisco_Goya', 'Frida_Kahlo', 'Georges_Seurat', 'Giotto_di_Bondone', 'Gustav_Klimt', 'Gustave_Courbet', 'Henri_Matisse', 'Henri_Rousseau', 'Henri_de_Toulouse-Lautrec', 'Hieronymus_Bosch', 'Jackson_Pollock', 'Jan_van_Eyck', 'Joan_Miro', 'Kazimir_Malevich', 'Leonardo_da_Vinci', 'Marc_Chagall', 'Michelangelo', 'Mikhail_Vrubel', 'Pablo_Picasso', 'Paul_Cezanne', 'Paul_Gauguin', 'Paul_Klee', 'Peter_Paul_Rubens', 'Pierre-Auguste_Renoir', 'Piet_Mondrian', 'Pieter_Bruegel', 'Raphael', 'Rembrandt', 'Rene_Magritte', 'Salvador_Dali', 'Sandro_Botticelli', 'Titian', 'Vasiliy_Kandinskiy', 'Vincent_van_Gogh', 'William_Turner']
Found 8774 fi

In [ ]:
# preprocessing the image data using the ResNet50 preprocessing function
from keras.applications.resnet50 import preprocess_input

# Preprocess the data
train_dataset = train_dataset.map(lambda x, y: (preprocess_input(x), y))
val_dataset = val_dataset.map(lambda x, y: (preprocess_input(x), y))

In [ ]:
# displaying training and validation curves for a given metric (accuracy or loss)
import plotly.express as px

def display_curves(history, metric):
  df = pd.DataFrame(history.history[metric], columns=[metric])
  df['val_'+metric] = history.history['val_'+metric]
  fig = px.line(df, x= df.index+1, y= [metric, 'val_'+metric])
  fig.update_layout(xaxis_title='Epochs', yaxis_title=metric)
  fig.show()

In [ ]:
# defining a ResNet50 model without pre-trained weights
model1 = keras.applications.ResNet50(input_shape= IMAGE_SIZE + (3,),
                                     classes= NUM_CLASSES,
                                     weights= None)

In [ ]:
model1.summary()

Model: "resnet50"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1             │ (None, 224, 224, 3)    │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_pad (ZeroPadding2D) │ (None, 230, 230, 3)    │              0 │ input_layer_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_conv (Conv2D)       │ (None, 112, 112, 64)   │          9,472 │ conv1_pad[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_bn                  │ (None, 112, 112, 64)   │            256 │ conv1_conv[0][0]       │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_relu (Activation)   │ (None, 112, 112, 64)   │              0 │ conv1_bn[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ pool1_pad (ZeroPadding2D) │ (None, 114, 114, 64)   │              0 │ conv1_relu[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ pool1_pool (MaxPooling2D) │ (None, 56, 56, 64)     │              0 │ pool1_pad[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_conv       │ (None, 56, 56, 64)     │          4,160 │ pool1_pool[0][0]       │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_bn         │ (None, 56, 56, 64)     │            256 │ conv2_block1_1_conv[0… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_relu       │ (None, 56, 56, 64)     │              0 │ conv2_block1_1_bn[0][… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_conv       │ (None, 56, 56, 64)     │         36,928 │ conv2_block1_1_relu[0… │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_bn         │ (None, 56, 56, 64)     │            256 │ conv2_block1_2_conv[0… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_relu       │ (None, 56, 56, 64)     │              0 │ conv2_block1_2_bn[0][… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_0_conv       │ (None, 56, 56, 256)    │         16,640 │ pool1_pool[0][0]       │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_3_conv       │ (None, 56, 56, 256)    │         16,640 │ conv2_block1_2_relu[0… │
│ (Conv2D)             

 Total params: 23,692,211 (90.38 MB)

 Trainable params: 23,639,091 (90.18 MB)

 Non-trainable params: 53,120 (207.50 KB)

In [ ]:
# starting with the ResNet50 architecture without pre-trained weights to evaluate how well it learns from the available data
model1.compile(optimizer = 'adam',
               loss = 'categorical_crossentropy',
               metrics = ['accuracy'])

checkpoint_cb = keras.callbacks.ModelCheckpoint("model.keras", save_best_only=True, monitor="val_loss")
early_stopping_cb = keras.callbacks.EarlyStopping(monitor='val_loss', patience=4)

EPOCHS = 15
history1 = model1.fit(train_dataset,
                     validation_data = val_dataset,
                     epochs = EPOCHS,
                     callbacks=[checkpoint_cb, early_stopping_cb])

Epoch 1/15
110/110 ━━━━━━━━━━━━━━━━━━━━ 170s 992ms/step - accuracy: 0.1418 - loss: 4.1054 - val_accuracy: 0.2395 - val_loss: 3.5577
Epoch 2/15
110/110 ━━━━━━━━━━━━━━━━━━━━ 94s 757ms/step - accuracy: 0.1917 - loss: 3.4323 - val_accuracy: 0.1619 - val_loss: 3.4524
Epoch 3/15
110/110 ━━━━━━━━━━━━━━━━━━━━ 88s 795ms/step - accuracy: 0.2517 - loss: 2.8245 - val_accuracy: 0.3769 - val_loss: 2.7746
Epoch 4/15
110/110 ━━━━━━━━━━━━━━━━━━━━ 87s 786ms/step - accuracy: 0.2863 - loss: 2.6437 - val_accuracy: 0.3483 - val_loss: 3.4263
Epoch 5/15
110/110 ━━━━━━━━━━━━━━━━━━━━ 89s 804ms/step - accuracy: 0.3075 - loss: 2.5195 - val_accuracy: 0.2360 - val_loss: 2.6836
Epoch 6/15
110/110 ━━━━━━━━━━━━━━━━━━━━ 82s 736ms/step - accuracy: 0.3473 - loss: 2.3689 - val_accuracy: 0.3381 - val_loss: 2.6849
Epoch 7/15
110/110 ━━━━━━━━━━━━━━━━━━━━ 141s 736ms/step - accuracy: 0.3778 - loss: 2.2456 - val_accuracy: 0.1773 - val_loss: 3.4965
Epoch 8/15
110/110 ━━━━━━━━━━━━━━━━━━━━ 86s 778ms/step - accuracy: 0.3856 - loss:

In [ ]:
display_curves(history1, 'loss')

In [ ]:
display_curves(history1, 'accuracy')

In [ ]:
# using pre-trained weights this time and freezing the entire model, only adding a few layers to the end
model2 = keras.applications.ResNet50(input_shape= IMAGE_SIZE + (3,),
                                     weights= 'imagenet',
                                     include_top= False,
                                     pooling= 'avg')
model2.trainable = False
x = keras.layers.Flatten()(model2.output)
x = keras.layers.Dense(512,activation='relu')(x)
x = keras.layers.Dropout(0.3)(x)
x = keras.layers.Dense(NUM_CLASSES, activation='softmax')(x)
model2 = keras.models.Model(model2.input, x)

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
model2.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2             │ (None, 224, 224, 3)    │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_pad (ZeroPadding2D) │ (None, 230, 230, 3)    │              0 │ input_layer_2[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_conv (Conv2D)       │ (None, 112, 112, 64)   │          9,472 │ conv1_pad[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_bn                  │ (None, 112, 112, 64)   │            256 │ conv1_conv[0][0]       │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_relu (Activation)   │ (None, 112, 112, 64)   │              0 │ conv1_bn[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ pool1_pad (ZeroPadding2D) │ (None, 114, 114, 64)   │              0 │ conv1_relu[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ pool1_pool (MaxPooling2D) │ (None, 56, 56, 64)     │              0 │ pool1_pad[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_conv       │ (None, 56, 56, 64)     │          4,160 │ pool1_pool[0][0]       │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_bn         │ (None, 56, 56, 64)     │            256 │ conv2_block1_1_conv[0… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_relu       │ (None, 56, 56, 64)     │              0 │ conv2_block1_1_bn[0][… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_conv       │ (None, 56, 56, 64)     │         36,928 │ conv2_block1_1_relu[0… │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_bn         │ (None, 56, 56, 64)     │            256 │ conv2_block1_2_conv[0… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_relu       │ (None, 56, 56, 64)     │              0 │ conv2_block1_2_bn[0][… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_0_conv       │ (None, 56, 56, 256)    │         16,640 │ pool1_pool[0][0]       │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_3_conv       │ (None, 56, 56, 256)    │         16,640 │ conv2_block1_2_relu[0… │
│ (Conv2D)             

 Total params: 24,662,963 (94.08 MB)

 Trainable params: 1,075,251 (4.10 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

In [ ]:
# compiling model2 and setting up callbacks for checkpointing and early stopping
model2.compile(optimizer = 'adam',
               loss = 'categorical_crossentropy',
               metrics = ['accuracy'])

checkpoint_cb = keras.callbacks.ModelCheckpoint("model2.keras", save_best_only=True, monitor="val_loss")
early_stopping_cb = keras.callbacks.EarlyStopping(monitor='val_loss', patience=4)

EPOCHS = 15
history2 = model2.fit(train_dataset,
                     validation_data = val_dataset,
                     epochs = EPOCHS,
                     callbacks=[checkpoint_cb, early_stopping_cb])

Epoch 1/15
110/110 ━━━━━━━━━━━━━━━━━━━━ 94s 730ms/step - accuracy: 0.2353 - loss: 3.2180 - val_accuracy: 0.6174 - val_loss: 1.4759
Epoch 2/15
110/110 ━━━━━━━━━━━━━━━━━━━━ 127s 655ms/step - accuracy: 0.5143 - loss: 1.7076 - val_accuracy: 0.7201 - val_loss: 1.0844
Epoch 3/15
110/110 ━━━━━━━━━━━━━━━━━━━━ 87s 703ms/step - accuracy: 0.6068 - loss: 1.3207 - val_accuracy: 0.7930 - val_loss: 0.8099
Epoch 4/15
110/110 ━━━━━━━━━━━━━━━━━━━━ 83s 708ms/step - accuracy: 0.6755 - loss: 1.0764 - val_accuracy: 0.8369 - val_loss: 0.6675
Epoch 5/15
110/110 ━━━━━━━━━━━━━━━━━━━━ 77s 702ms/step - accuracy: 0.7292 - loss: 0.8567 - val_accuracy: 0.8512 - val_loss: 0.5758
Epoch 6/15
110/110 ━━━━━━━━━━━━━━━━━━━━ 81s 695ms/step - accuracy: 0.7754 - loss: 0.7097 - val_accuracy: 0.8911 - val_loss: 0.4267
Epoch 7/15
110/110 ━━━━━━━━━━━━━━━━━━━━ 77s 700ms/step - accuracy: 0.8075 - loss: 0.6136 - val_accuracy: 0.9202 - val_loss: 0.3567
Epoch 8/15
110/110 ━━━━━━━━━━━━━━━━━━━━ 78s 705ms/step - accuracy: 0.8277 - loss: 

In [ ]:
display_curves(history2, 'loss')

In [ ]:
display_curves(history2, 'accuracy')

In [ ]:
# loading the best model from the saved file and evaluating its performance on both training and validation datasets
best_model = keras.models.load_model("model2.keras")

train_loss, train_accuracy = best_model.evaluate(train_dataset)
print(f"Training dataset - Loss: {train_loss:.4f}, Accuracy: {train_accuracy:.4f}")

val_loss, val_accuracy = best_model.evaluate(val_dataset)
print(f"Validation dataset - Loss: {val_loss:.4f}, Accuracy: {val_accuracy:.4f}")

110/110 ━━━━━━━━━━━━━━━━━━━━ 68s 524ms/step - accuracy: 0.9553 - loss: 0.1054
Training dataset - Loss: 0.1004, Accuracy: 0.9608
28/28 ━━━━━━━━━━━━━━━━━━━━ 17s 606ms/step - accuracy: 0.9365 - loss: 0.2875
Validation dataset - Loss: 0.2243, Accuracy: 0.9447
